In [ ]:
!pip install bitsandbytes>=0.41.3
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets
!pip uninstall torch torchdata -y
!pip install torch==1.13.1
!pip install torchdata==0.5.1
!pip install git+https://github.com/ludwig-ai/ludwig.git@master --quiet
!pip install bert-score
!pip install ludwig
!pip install nlpaug

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
ludwig 0.10.3.dev0 requires pyarrow<15.0.0, but you have pyarrow 16.1.0 which is incompatible.
Found existing installation: torch 2.3.0
Uninstalling torch-2.3.0:
  Successfully uninstalled torch-2.3.0
Found existing ins

  Using cached torchdata-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.6 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Operation cancelled by user


In [ ]:
import os
import gc

from google.colab import data_table

import numpy as np
import pandas as pd

import torch
from torch import Tensor

import datasets
from datasets import Dataset, DatasetDict

import yaml
from ludwig.api import LudwigModel, TrainingResults

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

file_name = "Output.csv"
try:
    # Prova a leggere il file con il delimitatore corretto
    df = pd.read_csv(file_name, delimiter=',', on_bad_lines='warn')
except pd.errors.ParserError as e:
    print(f"Error parsing CSV file: {e}")

# Rimuove il carattere `;` dal nome della colonna
df.columns = [col.strip(';') for col in df.columns]

# Visualizza le prime righe del dataset
print(df.head())

In [ ]:
# Pulizia del dataset
df_cleaned = df.dropna()

In [ ]:
from sklearn.utils import resample
import nlpaug.augmenter.word as naw

# Bilanciamento del dataset
min_class_size = df_cleaned['DOMANDE'].value_counts().min()
df_balanced = df_cleaned.groupby('DOMANDE').apply(lambda x: resample(x, replace=False, n_samples=min_class_size)).reset_index(drop=True)

# Augmentazione dei dati
aug = naw.SynonymAug(aug_src='wordnet')

df_balanced['DOMANDE_aug'] = df_balanced['DOMANDE'].apply(lambda x: aug.augment(x))
df_balanced['RISPOSTE_aug'] = df_balanced['RISPOSTE'].apply(lambda x: aug.augment(x))

df_augmented = pd.concat([df_balanced[['DOMANDE', 'RISPOSTE']],
                          df_balanced[['DOMANDE_aug', 'RISPOSTE_aug']].rename(columns={'DOMANDE_aug': 'DOMANDE', 'RISPOSTE_aug': 'RISPOSTE'})])

In [ ]:
df_augmented['DOMANDE'] = df_augmented['DOMANDE'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
df_augmented['RISPOSTE'] = df_augmented['RISPOSTE'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

In [ ]:
#suddivisione del set di dati in un set di addestramento, in uno di test e in uno di validazione
from sklearn.model_selection import train_test_split

#separo le caratteristiche dal target (divido input da output)
df_train, df_temp = train_test_split(df_cleaned, test_size=0.3, random_state=200)
df_test, df_validation = train_test_split(df_temp, test_size=0.1, random_state=200)

In [ ]:
# Verifica delle dimensioni
print(f'Train set size: {df_train.shape}')
print(f'Test set size: {df_test.shape}')
print(f'Validation set size: {df_validation.shape}')

In [ ]:
# Aggiungere una colonna per indicare lo split (per indicare quale riga appartiene a quale suddivisione.)
df_train["split"] = np.zeros(df_train.shape[0])
df_test["split"] = np.ones(df_test.shape[0])
df_validation["split"] = np.full(df_validation.shape[0], 2)

df_dataset = pd.concat([df_train, df_test, df_validation])
df_dataset["split"] = df_dataset["split"].astype(int)

# Verifica delle dimensioni complessive
print(df_dataset.shape)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Percorso alla cartella del modello salvato
model_path: str = "GroNLP/gpt2-small-italian"

# Carica il tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    trust_remote_code=True,
    padding_side="left",
    add_eos_token=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload", # Offload di parti del modello sulla CPU
    trust_remote_code=True,
    low_cpu_mem_usage=True,
   quantization_config=bnb_config
)

In [ ]:
# Estrazione delle prime 10 domande dal DataFrame originale
eval_df = df_test[['DOMANDE', 'RISPOSTE']].head(3)
# Estrai le risposte di riferimento
reference_responses = eval_df['RISPOSTE'].tolist()

In [ ]:
#Funzione per assicurarsi che le frasi termino con u punto
def ensure_ending_with_period(response):
    response = response.strip()
    if not response.endswith('.'):
       response += '.'
    return response

In [ ]:
prompt_template: str = """
#Genera una risposta per la seguente domanda, come se fossi un esperto cardiologo.
#Le risposte devono essere empatiche, di senso compiuto e LE FRASI DEVONO ESSERE LUNGHE MASSIMO 60 TOKEN E TERMINARE CON UN PUNTO.

### Domanda: {DOMANDE}

### Risposta:
"""

In [ ]:
from transformers import pipeline

# Funzione per generare risposte e calcolare BERTScore
def generate_responses_pre_finetuning(model, tokenizer, eval_df):
  #PROVO CON LA PIPELINE DI GENERAZIONE TESTO (Creo la pipeline di generazione)
  response_generator = pipeline(
      task="text-generation",
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.float16,
      device_map="auto"
  )

  # Genera e stampa la risposta per le prime 10 domande
  generated_responses = []
  for index, row in eval_df.iterrows():
    domande_test = row['DOMANDE']
    print(f'Generating response for question {index + 1}/{len(eval_df)}: {domande_test}')
    test_prompt = prompt_template.format(DOMANDE=domande_test)
    try:
      responses = response_generator(
          text_inputs=test_prompt,
          do_sample=True,
          top_k=50,
          num_return_sequences=1,
          eos_token_id=tokenizer.eos_token_id,
          max_length=512,
          truncation=True,
          #return_text=True
      )
      print(f'Pipeline output for question {index + 1}: {responses}')
      # Estrai e stampa la risposta generata
      generated_response = responses[0]['generated_text']
      generated_response = ensure_ending_with_period(generated_response)
      generated_responses.append(generated_response)
      print(f'\n[DOMANDA]: {domande_test}\n[GENERATED_TEXT] MODEL_PREDICTION:\n{generated_response}\n')
    except Exception as e:
      print(f"Error generating response for question {index + 1}: {e}")
    print(f"Generated {len(generated_responses)} responses out of {len(eval_df)} expected.")
  return generated_responses

In [ ]:
# le risposte con il modello pre-addestrato
print("Generating responses with the pre-fine-tuning model...")
generated_responses_pre = generate_responses_pre_finetuning(model, tokenizer, eval_df)

In [ ]:
from bert_score import score
# Calcola BERTScore per il modello pre-addestrato
print("Evaluating BERTScore for the pre-fine-tuning model...")
P_pre, R_pre, F1_pre = score(generated_responses_pre, reference_responses, lang="it", verbose=True)
print(f"Pre-fine-tuning BERTScore - Precision: {P_pre.mean().item()}, Recall: {R_pre.mean().item()}, F1: {F1_pre.mean().item()}")


In [ ]:
# Converti i tensori PyTorch in array NumPy, se necessario
if isinstance(P_pre, torch.Tensor):
    P_pre = P_pre.numpy()
if isinstance(R_pre, torch.Tensor):
    R_pre = R_pre.numpy()
if isinstance(F1_pre, torch.Tensor):
    F1_pre = F1_pre.numpy()

# Stampa i punteggi medi
print(f'Mean Precision: {np.mean(P_pre):.4f}')
print(f'Mean Recall: {np.mean(R_pre):.4f}')
print(f'Mean F1: {np.mean(F1_pre):.4f}')

In [ ]:
model.gradient_checkpointing_enable()
# Free up memory
torch.cuda.empty_cache()
gc.collect()

In [ ]:
yaml_config_template = """
model_type: llm
base_model: {model_path}

input_features:
  - name: DOMANDE
    type: text
    preprocessing:
      max_sequence_length: 300
      truncation: true
      lowercase: true

output_features:
  - name: RISPOSTE
    type: text
    preprocessing:
      max_sequence_length: 300
      truncation: true
    metrics:
      - type: bert_score
        model: dbmdz/bert-base-italian-uncased
      - type: rouge
      - type: perplexity

prompt:
  template: >-
    Genera una risposta per la seguente domanda, come se fossi un esperto cardiologo.
    Le risposte devono essere empatiche, di senso compiuto e LE FRASI DEVONO TERMINARE CON UN PUNTO e essere lunghe massimo 60 token.
    ### Domanda: {DOMANDE}

    ### Risposta (Deve terminare con un punto e essere di senso compiuto):

generation:
  temperature: 0.6
  max_new_tokens: 60
  top_p: 0.9

adapter:
  type: lora

quantization:
  bits: 4

preprocessing:
  split:
    type: fixed

trainer:
  type: finetune
  epochs: 50
  batch_size: 16
  eval_batch_size: 8
  gradient_accumulation_steps: 2
  learning_rate: 2.0e-5
  enable_gradient_checkpointing: true
  weight_decay: 0.01
  dropout_rate: 0.1
  learning_rate_scheduler:
    decay: cosine
    warmup_fraction: 0.1
    reduce_on_plateau: 0
  checkpoint_interval: 500

early_stopping:
  monitor: val_loss
  patience: 5
  mode: min

logging:
  log_to_tensorboard: true

callbacks:
  - type: progress_bar
  - type: model_checkpoint
    save_best_only: true
    monitor: val_loss
    mode: min
  - type: early_stopping
    monitor: val_loss
    patience: 5
    mode: min
  """

In [ ]:
# Sostituisci solo il segnaposto {model_path}
yaml_config = yaml_config_template.replace("{model_path}", model_path)

# Carica la configurazione YAML come dizionario
qlora_fine_tuning_config = yaml.safe_load(yaml_config)

In [ ]:
# Inizializza il modello Ludwig con la configurazione aggiornata
model_ludwig = LudwigModel(config=qlora_fine_tuning_config)

In [ ]:
# Esegui il fine-tuning del modello con Ludwig
model_ludwig.train(dataset=df_train)

In [ ]:
#Lo salvo e lo ricarico per eseguirlo
# Salva il modello
model_ludwig.save("FineTuned_GPT2_Ludwig")

In [ ]:
# Carica il modello salvato
model_loaded = LudwigModel.load("FineTuned_GPT2_Ludwig")

In [ ]:
from bert_score import score

# Funzione per generare risposte e calcolare BERTScore
def generate_responses_post_finetuning(model, eval_df):
    # Ottieni le previsioni dal modello
    predictions_df = model.predict(dataset=eval_df)

    # Se predictions_df è una tupla, ottieni il DataFrame delle previsioni
    if isinstance(predictions_df, tuple):
        predictions_df = predictions_df[0]

    # Verifica se il DataFrame predictions contiene la colonna 'RISPOSTE_response'
    if 'RISPOSTE_response' in predictions_df.columns:
        # Estrai le risposte generate
        generated_responses = predictions_df['RISPOSTE_response'].apply(lambda x: ''.join(x) if isinstance(x, list) else x).tolist()

        for domanda, risposta in zip(eval_df['DOMANDE'], generated_responses):
            print(f'\n[DOMANDA]: {domanda}\n[GENERATED_TEXT] MODEL_PREDICTION:\n{risposta}\n')
        return generated_responses
    else:
        raise ValueError("La colonna 'RISPOSTE_response' non è presente nel DataFrame delle previsioni.")


In [ ]:
# Genera le risposte con il modello fine-tunato
print("Generating responses with the post-fine-tuning model...")
generated_responses_post = generate_responses_post_finetuning(model_loaded, eval_df)


In [ ]:
from bert_score import score
# Calcola BERTScore per il modello fine-tunato
print("Evaluating BERTScore for the post-fine-tuning model...")
P_post, R_post, F1_post = score(generated_responses_post, reference_responses, lang="it", verbose=True)
print(f"Post-fine-tuning BERTScore - Precision: {P_post.mean().item()}, Recall: {R_post.mean().item()}, F1: {F1_post.mean().item()}")

In [ ]:
# Converti i tensori PyTorch in array NumPy, se necessario
if isinstance(P_post, torch.Tensor):
    P = P_post.numpy()
if isinstance(R_post, torch.Tensor):
    R = R_post.numpy()
if isinstance(F1_post, torch.Tensor):
    F1 = F1_post.numpy()

# Stampa i punteggi medi
print(f'Mean Precision: {np.mean(P):.4f}')
print(f'Mean Recall: {np.mean(R):.4f}')
print(f'Mean F1: {np.mean(F1):.4f}')

In [ ]:
# Ottieni le previsioni dal modello
df_pred = model_loaded.predict(dataset=eval_df)

# Se df_pred è una tupla, ottieni il DataFrame delle previsioni
if isinstance(df_pred, tuple):
   df_pred = df_pred[0]

In [ ]:
import subprocess
# Avvia TensorBoard in background
subprocess.Popen(['tensorboard', '--logdir', 'results'])

#FILE JSON:
Il file JSON che hai fornito contiene diverse informazioni riguardanti il processo di training del modello. Ecco un'analisi dettagliata dei principali componenti del file:

Struttura del File JSON
1. evaluation_frequency:
  * frequency: Indica la frequenza con cui viene effettuata la valutazione (1 in questo caso, che probabilmente significa ogni epoca).
  * period: Indica il periodo della valutazione, in questo caso "epoch".

2. test: questo campo è vuoto, indicando che non ci sono dati di test disponibili nel file.

3. training: contiene informazioni dettagliate sul processo di training per la feature RISPOSTE.

  * RISPOSTE:
    * loss: Una lista di valori di loss registrati durante il training per ogni epoca.
    * next_token_perplexity: Una lista di valori di next_token_perplexity registrati durante il training per ogni epoca.
    * perplexity: Una lista di valori di perplexity registrati durante il training per ogni epoca.
    * sequence_accuracy: Una lista di valori di sequence_accuracy registrati durante il training per ogni epoca.
    * token_accuracy: Una lista di valori di token_accuracy registrati durante il training per ogni epoca.
  * combined:
    * loss: Una lista di valori di loss combinata registrati durante il training per ogni epoca.
4. validation: questo campo è vuoto, indicando che non ci sono dati di validazione disponibili nel file.

##Interpretazione delle Metriche
* loss: La funzione di perdita (loss) misura quanto bene il modello sta facendo durante il training. Valori più bassi indicano un modello migliore.
* next_token_perplexity: La perplexity del prossimo token è una misura della probabilità di una sequenza. Valori più bassi indicano che il modello prevede meglio le sequenze.
* perplexity: La perplexity misura quanto bene il modello prevede una sequenza di dati. Valori più bassi indicano prestazioni migliori.
* sequence_accuracy: La sequence_accuracy misura la percentuale di sequenze completamente corrette previste dal modello.
* token_accuracy: La token_accuracy misura la percentuale di token correttamente previsti dal modello.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import json

# Carica i dati del log
log_path = 'results/api_experiment_run/training_statistics.json'

with open(log_path, 'r') as f:
    log_data = json.load(f)

# Verifica la struttura dei dati
print(json.dumps(log_data, indent=2))

In [ ]:
training_loss = log_data['training']['RISPOSTE']['loss']
next_token_perplexity = log_data['training']['RISPOSTE']['next_token_perplexity']
token_accuracy = log_data['training']['RISPOSTE']['token_accuracy']

# Crea un DataFrame con i dati estratti
df_metrics = pd.DataFrame({
    'epoch': range(1, len(training_loss) + 1),
    'train_loss': training_loss,
    'next_token_perplexity': next_token_perplexity,
    'token_accuracy': token_accuracy
})

In [ ]:
# Grafico per la loss
plt.figure(figsize=(10, 6))
plt.plot(df_metrics['epoch'], df_metrics['train_loss'], label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Loss')
plt.show()

# Grafico per next_token_perplexity
plt.figure(figsize=(10, 6))
plt.plot(df_metrics['epoch'], df_metrics['next_token_perplexity'], label='Next Token Perplexity')
plt.xlabel('Epoch')
plt.ylabel('Next Token Perplexity')
plt.legend()
plt.title('Next Token Perplexity')
plt.show()

# Grafico per token_accuracy
plt.figure(figsize=(10, 6))
plt.plot(df_metrics['epoch'], df_metrics['token_accuracy'], label='Token Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Token Accuracy')
plt.legend()
plt.title('Token Accuracy')
plt.show()

#CONFIGURAZIONE YAML PER VALUTAZIONE GRAFICI:
yaml_config_template = """

model_type: llm

base_model: {model_path}


input_features:

  - name: DOMANDE

    type: text

    preprocessing:

      max_sequence_length: 300

      truncation: true     

      lowercase: true  

output_features:

  - name: RISPOSTE

    type: text

    preprocessing:

      max_sequence_length: 300

      truncation: true

    metrics:

      - type: bert_score

        model: dbmdz/bert-base-italian-uncased

      - type: rouge  # Aggiungi metriche ROUGE per valutare meglio le risposte generate

      - type: perplexity  # Aggiungi perplexity come metrica

prompt:

  template: >-

    Genera una risposta per la seguente domanda, come se fossi un esperto cardiologo.
    Le risposte devono essere empatiche, di senso compiuto e LE FRASI DEVONO TERMINARE CON UN PUNTO e essere lunghe massimo 60 token.
    ### Domanda: {DOMANDE}

    ### Risposta (Deve terminare con un punto e essere di senso compiuto):

generation:

  temperature: 0.6  # Abbassa la temperatura per risposte più coerenti

  max_new_tokens: 60

  top_p: 0.9  # Aggiungi top_p per migliorare la qualità delle risposte


adapter:

  type: lora


quantization:

  bits: 4


preprocessing:

  split:

    type: fixed


trainer:

  type: finetune

  epochs: 100  # Aumenta il numero di epoche per un fine-tuning più approfondito
  
  batch_size: 8  # Aumenta la dimensione del batch se la memoria lo permette
  
  eval_batch_size: 4
  
  gradient_accumulation_steps: 2  # Regola in base alla memoria disponibile
  
  learning_rate: 3.0e-5  # Prova con un learning rate più basso
  
  enable_gradient_checkpointing: true
  
  weight_decay: 0.01  # Aggiungi weight decay per regolarizzare
  
  dropout_rate: 0.1  # Aggiungi dropout per regolarizzare
  
  learning_rate_scheduler:
  
    decay: cosine
  
    warmup_fraction: 0.1  # Mantieni il warmup fraction
  
    reduce_on_plateau: 0
  
  checkpoint_interval: 500  # Salva un checkpoint ogni 500 steps


early_stopping:

  monitor: val_loss

  patience: 3

  mode: min


logging:

  log_to_tensorboard: true


callbacks:

  - type: progress_bar

  - type: model_checkpoint

    save_best_only: true

    monitor: val_loss

    mode: min

  - type: early_stopping

    monitor: val_loss

    patience: 3

    mode: min

  """

#COMMENTO DEI GRAFICI:
##TOKEN ACCURACY:
Mostra l'andamento dell'accuratezza dei token (Token Accuracy) durante le epoche di addestramento del tuo modello. L'asse X rappresenta il numero di epoche, mentre l'asse Y rappresenta l'accuratezza dei token.
* DECLINO PROGRESSIVO:
L'accuratezza dei token inizia a circa 0.06 (6%) e diminuisce progressivamente fino a circa 0.02 (2%) verso la fine del training.
Questo declino indica che il modello sta diventando meno accurato nel prevedere i token corretti man mano che l'addestramento procede.
>>> Possibile OVERFITTING O PROBLEMI DI ADDESTRAMENTO:
Un declino continuo e significativo nell'accuratezza dei token può indicare che il modello sta sovraccaricando il rumore presente nel set di dati di training, o potrebbe esserci un problema nella configurazione dell'addestramento.
Un'altra possibilità è che il modello stia incontrando difficoltà con il set di dati e non stia apprendendo in modo efficace.

**SOLUZIONI:**
1. Verifica del Set di Dati:
Assicurati che il set di dati di training sia di alta qualità e bilanciato. Dati rumorosi o squilibrati possono influire negativamente sull'addestramento del modello.

2. Regolazione dei Parametri di Addestramento:
Prova a regolare l'ipotesi di apprendimento (learning rate), la
dimensione del batch, e altre iperparametri per migliorare le prestazioni del modello.
Potresti voler aggiungere tecniche di regolarizzazione come il dropout per prevenire l'overfitting.

3. Early Stopping:
Implementa l'early stopping per interrompere l'addestramento se l'accuratezza dei token continua a diminuire, evitando così l'overfitting.

5. Analisi dei Modelli di Perdita:
Oltre all'accuratezza dei token, osserva anche l'andamento della perdita (loss). Un'analisi combinata delle metriche può fornire ulteriori indicazioni su come migliorare l'addestramento del modello.

##NEXT TOKEN PERPLEXITY
L'asse X rappresenta il numero di epoche, mentre l'asse Y rappresenta la perplexity del prossimo token. La perplexity misura quanto bene il modello prevede la sequenza di token. Valori più bassi di perplexity indicano una migliore capacità di previsione.

* DECLINO PROGRESSIVO: La perplexity inizia intorno a 28500 e diminuisce progressivamente fino a circa 26000 verso la fine del training. Questo declino indica che il modello sta migliorando nella previsione dei token successivi man mano che l'addestramento procede.
Il modello sta diventando più preciso nelle sue previsioni.

Confrontando questo grafico con quello dell'accuratezza dei token che hai fornito in precedenza, notiamo un contrasto interessante:
Accuratezza dei Token: Mostra un declino costante, suggerendo che il modello sta diventando meno preciso nel prevedere i token corretti.
Next Token Perplexity: Mostra un declino, suggerendo che il modello sta migliorando nella previsione delle sequenze.

##TRAINING LOSS
mostra l'andamento della perdita di addestramento (Training Loss) durante le epoche di addestramento del tuo modello. L'asse X rappresenta il numero di epoche, mentre l'asse Y rappresenta il valore della perdita.

* DECLINO PROGRESSIVO: La perdita inizia intorno a 7.5 e diminuisce progressivamente fino a circa 4.0 verso la fine del training. Questo declino indica che il modello sta migliorando nell'addestramento, riducendo l'errore nelle sue previsioni.
Indica che il modello sta imparando dai dati di training e sta migliorando nel tempo.

Considera l'aggiunta di tecniche di regolarizzazione come il dropout o la weight decay per prevenire l'overfitting e migliorare la generalizzazione del modello.

In [ ]:
# Comprimi la directory del modello
!zip -r FineTuned_GPT_Ludwig.zip FineTuned_GPT2_Ludwig

# Scarica il file compresso sul tuo PC
files.download('FineTuned_GPT2_Ludwig.zip')

In [ ]:
# Salva i risultati in un nuovo file CSV
results_dir = 'results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Percorso del file da salvare
file_path = os.path.join(results_dir, 'bertscore_results1.csv')

df_pred.to_csv(file_path, index=False)

import shutil

# Percorso del file zip
zip_file = 'results.zip'

# Comprime la cartella 'results'
shutil.make_archive('results', 'zip', 'results')

print(f"Cartella compressa in: {zip_file}")

# Scarica il file zip
files.download(zip_file)